
<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 10px">

#  API Demo / Lab + NLP


<img src="https://snag.gy/RNAEgP.jpg" width="600">

Can we correctly identify which of these two old men tweeted what?


## (5 mins) Opening 

We are going to:

- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API

(Lab)
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

![](https://snag.gy/HPBQbJ.jpg)

We now will now go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).  After we set up our app, we will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

Someone was nice enough to build a nice libary for us in Python that we only need to plug in our keys and start collecting data with.  The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [5]:
!pip install python-twitter

## Some Boring Twitter Rules

Twitter says they will rate limit your requests:

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rulez":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys

Take note of our application keys that we will be using with our little application that will be connecting to Twitter and mining Tweets from the official twitter accounts of our choosing.


## Tweet Miner Class Setup

The following code is meant to get us up and running with connectivity to twitter, and the ability to make requests and easily transform the JSON responses to DataFrames.  We will be using object oriented Python in order to organize our code.  We may go into review since this was a topic we covered earlier in the class but we can review it during the lab for those who want to know more about it.


> "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding any rate limit blocks.

#### Key Setup

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the "generate tokens" button to get this
- **access_token_secret** - Also available after "generate tokens" is pressed


In [3]:
import twitter, re, datetime, pandas as pd

class twitterminer():

    request_limit   =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        '2UMx1b3sVexoau3RBMqDkCnQk',
        'consumer_secret':     'B30hlQ260D6ZEuu1FavRo0ygz78VPiQU5T9WqhqplWrqSejX6u',
        'access_token_key':    '441431405-G7NP80h5k308x6qSHc8nOpTJ7p4wa6YG5Oz66uVf',
        'access_token_secret': 'BTK48YLcRfCO9ldaYawiEpeGjuA0bySMYbnio0f1OHciD'
    }
    
    def __init__(self,  request_limit = 20):
        
        self.request_limit = request_limit
        
        # This sets the twitter API object for use internall within the class
        self.set_api()
        
    def set_api(self):
        
        self.api = twitter.Api(
            consumer_key         =   self.twitter_keys['consumer_key'],
            consumer_secret      =   self.twitter_keys['consumer_secret'],
            access_token_key     =   self.twitter_keys['access_token_key'],
            access_token_secret  =   self.twitter_keys['access_token_secret']
        )

    def mine_user_tweets(self, user="matthewbrems"):

        statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.request_limit)
        data       =   []
        
        for item in statuses:

            mined = {
                'tweet_id': item.id,
                'handle': item.user.name,
                'retweet_count': item.retweet_count,
                'text': item.text,
                'mined_at': datetime.datetime.now(),
                'created_at': item.created_at,
            }
            
            data.append(mined)
            
        return data

## Does anyone remember how we "instantiate" a new instance of this class?

**Bonus bonus** How do we call the method to *mine_user_tweets()*?

In [44]:
# optional twitter ids:  realDonaldTrump, berniesanders
# Let's test this out here..

##  Now we create some training data

We will have to munge a little bit in order to get our "mined" data from the Twitter API.  

 - Mine User1 Tweets
 - Create DataFrame
 - Mine User2 Tweets
 - Append to DataFrame

In [4]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = twitterminer(request_limit=100)

In [5]:
# insert handle we like
brems_tweets = miner.mine_user_tweets()

In [7]:
# name our df aptly
brems_df = pd.DataFrame(brems_tweets)
brems_df.head(10)

/Users/JosephNelson/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/JosephNelson/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/JosephNelson/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/JosephNelson/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/JosephNelson/anaconda2/

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Jan 25 16:22:57 +0000 2017,Matt Brems,2017-01-25 11:40:23.086171,13,RT @jacobsoboroff: True. https://t.co/GWq6DjedD1,824291478784589824
1,Wed Jan 25 16:22:32 +0000 2017,Matt Brems,2017-01-25 11:40:23.086184,43833,RT @mightybattlecat: The year is 2017. America...,824291372664520706
2,Wed Jan 25 14:50:53 +0000 2017,Matt Brems,2017-01-25 11:40:23.086188,17444,RT @ericgeller: It's happening: The White Hous...,824268309717721088
3,Wed Jan 25 14:46:54 +0000 2017,Matt Brems,2017-01-25 11:40:23.086192,0,S8 of Parks and Rec?? https://t.co/UzvK4W6jbu,824267305504874496
4,Wed Jan 25 14:46:31 +0000 2017,Matt Brems,2017-01-25 11:40:23.086197,750,RT @CahnEmily: Your daily reminder that @realD...,824267209295929344
5,Wed Jan 25 14:46:16 +0000 2017,Matt Brems,2017-01-25 11:40:23.086201,0,#DataScience https://t.co/VvYnjsdpvH,824267147069243392
6,Wed Jan 25 03:43:03 +0000 2017,Matt Brems,2017-01-25 11:40:23.086206,306,RT @HalfOnionInABag: @Aaron_Spicer @realDonald...,824100243990056962
7,Wed Jan 25 03:41:46 +0000 2017,Matt Brems,2017-01-25 11:40:23.086210,0,"@mostlyinane Missed you tonight, but glad it w...",824099917476073472
8,Tue Jan 24 21:45:01 +0000 2017,Matt Brems,2017-01-25 11:40:23.086213,0,😂😂 https://t.co/NjVd1Vpcwx,824010141553094656
9,Tue Jan 24 17:05:20 +0000 2017,Matt Brems,2017-01-25 11:40:23.086219,19689,RT @AdamWeinstein: Is that... a problem\nHas.....,823939757273206784


In [8]:
brems_df.shape

(100, 6)

## Any interesting ngrams going on with Trump?

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,2))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(10)

[(u'https co', 31),
 (u'thank you', 14),
 (u'crooked hillary', 10),
 (u'of the', 10),
 (u'trump2016 https', 8),
 (u'in the', 8),
 (u'will be', 6),
 (u'great again', 5),
 (u'america great', 5),
 (u'for the', 5)]

## (10 mins) Try this exercize with Bernie Sanders..

In [103]:
sanders_tweets = miner.mine_user_tweets("berniesanders")

In [104]:
all_tweets = pd.DataFrame(trump_tweets + sanders_tweets)

## Preprocessing our Tweets

In order to do classfication recall that we need a set of features.  Our features are literally what our presidential hopefulls say on Twitter. 

We will need to:
- Vectorize input text data
- Intialize a model (let's try Logistic regression)
- Train / Predict / Cross Validate
- Score / Evaluate


In [107]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(lowercase=True, strip_accents='unicode')
X_all = tfv.fit_transform(all_tweets['text'])

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X_all, all_tweets['handle'])



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Check Prediction vs Random Sanders Tweet

In [110]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

X_all = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(X_all)

array([[ 0.63681826,  0.36318174],
       [ 0.29723894,  0.70276106]])

## Lab Time

### The following is available in the lab folder as well -- please do your work in _that_ notebook.

We would like you to perform an analysis using a proper cross validation.  Also, try classfication using other models.

### 1. Implement the same analysis using more data.

Experiment with using more data.  The API may not like that you are blowing through their limits so definitely be careful.  Try to grab only what you need 1x, then work on the copy of the objects that are returned.  Read the documents about rate limits and see if you can get enough without hitting the rate limit.  Are there any options availabl in the API to avoid such a problem?

### 2. Implement K-Folds or test/train split.

Double check that you are getting random data before moving forward.  What would happen if you over sample Trump more than Sanders?

### 3. Mine more Tweets that aren't in your data set
Or use the hold-out method to do a proper test.  Refer back to our advanced classification evaluation lesson if you need to.

### 4. Check your classification report
How's precision / recall of your model?

### 5.  Change out your TFIDF vectorizer for CountVectorizer.
How has this impacted your mode performance at all?

### 6.  Implement a different classification method such as random forrests.
Or pick one of your favorites

### 7.  Try to remove stopwords from your text during your preprocessing step

Then double check your classfication report.  Have things improved?

### 8.  Try removing samples that have links or that are obviously just announcements or "noise" that doesn't appear to represent "True" tweets by the authors.

### 9. What are some contrasting words or phrases that you can see between the ngrams for each author?

### 10.  What do you think you can do to improve the scores further?

### 11. **BONUS** Using TextBlob, add a sentiment feature to your dataset.

### 12. BONUS BONUS Apply PCA to your text features
Is this effective? (ie: we could talk about LDA here a little bit)

## Closing

- What where the most impactful changes that helped your models?
- What do you think would happen if we had more user1 Tweets than user2?
- What other projects might you think to apply these problems against?